In [1]:
from classes.py import *
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

ModuleNotFoundError: No module named 'kqc_custom'

In [3]:
def normal_encoding(data,wozero = True):
    result = []
    temp = data
    if wozero == True : temp = temp[temp!=0]
    mean = np.mean(temp); sd = np.std(temp)
    if sd ==0 : return(data)
    temp = (temp-mean)/sd5010105
    temp = temp.apply(stats.norm.cdf)
    result = pd.DataFrame(np.zeros(data.shape))
    result.iloc[:,0] = temp;result = result.replace(np.NaN,0)
    return(result)

def exponential_encoding(data,wozero = True):
    result = []
    temp = data
    if wozero == True : temp = temp[temp!=0]
    mean = np.mean(temp); sd = np.std(temp)
    if sd ==0 : return(data/10)
    temp = 1-np.exp(-(temp/mean).astype('float'))
    result = pd.DataFrame(np.zeros(data.shape))
    result.iloc[:,0] = temp;result = result.replace(np.NaN,0)
    return(result)
    
def order_encoding(data):
    n=len(data)
    temp = data;temp = temp[temp!=0]
    temp = pd.DataFrame(stats.rankdata(temp)/n, index = temp.index)
    result = pd.DataFrame(np.zeros(data.shape))
    result.iloc[:,0] = temp;result = result.replace(np.NaN,0)
    return(result)

def min_max_scaler(data): return (data-np.min(data))/(np.max(data)-np.min(data))

def Cross_val(X_temp,y,Model,k):
    n = X_temp.shape[0];p = X_temp.shape[1]
    m = int(n/k)
    train_list = []
    test_list = []
    for i in range(k):
        test_index = [j+m*i for j in range(m)]
        train_index = [j for j in range(0,np.min(test_index))]+[j for j in range(np.max(test_index),n)]
        X_train = X_temp.iloc[train_index,:]
        X_test  = X_temp.iloc[test_index,:]
        y_train = y[train_index]
        y_test  = y[test_index]
        Model.fit(X_train,y_train)
        train_list += [lr.score(X_train,y_train)]
        test_list += [lr.score(X_test,y_test)]
        
    result = pd.DataFrame([np.mean(train_list),np.mean(test_list)]).T.append(pd.DataFrame([train_list,test_list]).T)
    result.columns = ["Train Score","Test Score"]
    result.index = ["Average"]+[str(i+1)+"th Fold" for i in range(k)]
    return(result)

def numstr(num,r=2):return(str(round(num,2)))

def stand(x) : 
    dia = np.apply_along_axis(np.var,0,x)
    dia[dia!=0] = dia[dia!=0]**(-0.5)
    return(np.matmul(x-np.apply_along_axis(np.mean,0,x),np.diag(dia)))

def ridgepower(mat,power,eps):
    big = np.linalg.eig(mat)[0][0] 
    return(matpower(mat+np.diag(np.repeat(eps*big,mat.shape[0])),power))
    
def matpower(mat,power,thre = 0.00000001):
    mat = (mat + mat.T)/2
    eig = np.linalg.eig(mat)
    val = np.diag([np.real(i**power) if i >thre else 0 for i in eig[0]])
    vec = np.real(eig[1])
    return(np.matmul(np.matmul(vec,val),vec.T))

def SIR(y,X,h):
    n = X.shape[0];p = X.shape[1] 
    signrt = matpower(np.diag(np.var(X)),-0.5)
    X = stand(X)
    ylabel=[]
    ylabel = np.unique(np.array(y))
    prob = np.array([]);exy =np.array([np.repeat(0,p)])
    for i in range(h):
        prob = np.append(prob, y[np.array(y==ylabel[i])].shape[0]/n)
    for i in range(h):
        exy = np.concatenate([exy,[np.apply_along_axis(np.mean,0,X.iloc[np.array(y==ylabel[i]),:])]],axis=0)
    exy = exy[1:,:]
    sirmat = np.matmul(np.matmul(exy.T,np.diag(prob)),exy)
    eig = np.linalg.eigh(sirmat)
    return([np.matmul(signrt,eig[1]),eig[0]])

def SIRmat(y,X,h):
    n = X.shape[0];p = X.shape[1] 
    signrt = matpower(np.diag(np.var(X)),-0.5)
    X = stand(X)
    ylabel=[]
    ylabel = np.unique(np.array(y))
    prob = np.array([]);exy =np.array([np.repeat(0,p)])
    for i in range(h):
        prob = np.append(prob, y[np.array(y==ylabel[i])].shape[0]/n)
    for i in range(h):
        exy = np.concatenate([exy,[np.apply_along_axis(np.mean,0,X.iloc[np.array(y==ylabel[i]),:])]],axis=0)
    exy = exy[1:,:]
    sirmat = np.matmul(np.matmul(exy.T,np.diag(prob)),exy)
    return(sirmat)

def DR(y,X,h) :
    n = X.shape[0];p = X.shape[1] 
    signrt = matpower(np.diag(np.var(X)),-0.5)
    X = stand(X)
    yless = np.array(y);ylabel=[]
    ylabel = np.unique(np.array(y))
    prob = np.array([]);exy =np.array([np.repeat(0,p)])
    for i in range(h):
        prob = np.append(prob, y[np.array(y==ylabel[i])].shape[0]/n)
    vxy = np.zeros(shape=(p,p,h))
    for i in range(h):
        vxy[:,:,i] = np.cov(X.iloc[np.array(y==ylabel[i]),].T)
        exy = np.concatenate([exy,[np.apply_along_axis(np.mean,0,X.iloc[np.array(y==ylabel[i]),:])]],axis=0)
    exy = exy[1:,:]
    mat1 = np.zeros(shape=(p,p));mat2 = np.zeros(shape=(p,p))
    for i in range(h):
        xxt = np.matmul(exy[i:i+1,:].T,exy[i:i+1,:])
        mat1 = mat1+prob[i]*np.matmul((vxy[:,:,i]+xxt),vxy[:,:,i]+xxt)
        mat2 = mat2+prob[i]*xxt
    out = 2*mat1+2*np.matmul(mat2,mat2)+2*np.sum(np.diag(mat2))*mat2-2*np.identity(p)
    eig = np.linalg.eig(out)
    return([np.real(np.matmul(signrt,eig[1])),eig[0]])

In [4]:
n=500;p=20
X,y = kqc_custom.generate_dependent_sample(n,p,beta_coef=[5 for i in range(8)],epsilon=5,covariance_parameter=1)

In [5]:
lr = LinearRegression()
Cross_val(pd.DataFrame(X),y,lr,5)

,Train Score,Test Score
Average,0.937848,0.928910
1th Fold,0.941084,0.914962
2th Fold,0.938021,0.928368
3th Fold,0.942147,0.910331
4th Fold,0.933474,0.947732
5th Fold,0.934515,0.943157


In [6]:
sparsity = 80
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))

In [9]:
pd.DataFrame(X).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-2.222292,0.438688,-0.892582,0.801415,3.425105,1.194436,-1.660251,-3.550949,-1.730477,-0.677431,5.123301,-1.648834,1.975133,0.969462,-0.799817,0.941106,-3.282297,-2.485294,-2.325333,-0.001207
1,1.745869,2.554809,0.243340,-0.787218,1.833025,-0.553911,0.373287,0.318738,1.113856,-2.333281,2.002366,-1.573906,1.324312,-0.233500,-2.048148,-0.368853,2.190424,0.333533,1.806769,0.035200
2,1.176601,1.293326,-2.700726,2.952051,0.862495,-0.768287,-3.951313,2.084775,0.631244,1.068560,2.072137,-2.593286,-1.180815,-0.408574,2.468772,-1.911556,-1.257762,-3.263892,-0.023902,1.027020
3,2.646543,0.087070,0.329800,2.749217,-1.465542,0.793024,-0.398554,-1.406785,2.835931,-1.811529,2.400699,-2.224827,-1.805235,-0.842794,0.516911,-1.526379,-1.012912,-1.815651,-1.880951,-1.076060
4,0.859401,-1.001736,0.489184,-3.376846,-0.024978,-0.228031,-0.276048,-0.420998,-0.910101,0.720432,1.600718,-1.121185,-0.865701,0.947140,1.113404,0.088312,-1.544792,-1.153150,-2.512998,-1.804045


In [8]:
X_sparse.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-0.000000,0.000000,-0.000000,0.801415,0.0,0.0,-0.0,-0.000000,-1.730477,-0.677431,0.000000,-0.000000,0.000000,0.0,-0.000000,0.0,-3.282297,-0.00000,-0.000000,-0.0
1,0.000000,0.000000,0.000000,-0.000000,0.0,-0.0,0.0,0.000000,1.113856,-0.000000,0.000000,-0.000000,0.000000,-0.0,-0.000000,-0.0,0.000000,0.00000,0.000000,0.0
2,1.176601,1.293326,-0.000000,0.000000,0.0,-0.0,-0.0,0.000000,0.000000,0.000000,2.072137,-0.000000,-1.180815,-0.0,0.000000,-0.0,-0.000000,-0.00000,-0.023902,0.0
3,2.646543,0.000000,0.000000,2.749217,-0.0,0.0,-0.0,-1.406785,2.835931,-1.811529,0.000000,-0.000000,-0.000000,-0.0,0.516911,-0.0,-0.000000,-0.00000,-0.000000,-0.0
4,0.859401,-0.000000,0.489184,-0.000000,-0.0,-0.0,-0.0,-0.000000,-0.000000,0.000000,0.000000,-1.121185,-0.000000,0.0,1.113404,0.0,-1.544792,-1.15315,-0.000000,-0.0


In [10]:
n=500;p=5
X,y = kqc_custom.generate_dependent_sample(n,p,beta_coef=[5 for i in range(3)],epsilon=5,covariance_parameter=1)

In [15]:
sparsity = 20
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
oPCA = OrdinaryEig()
x_pca = pd.DataFrame(np.matmul(X_sparse.values,oPCA.fit_transform(X_sparse.corr()).values))
sEig = SparseEig(r=2,k=10)
x_spca = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
bEig = BinaryEig(r=2,k=[-0.5,0.3])
x_bpca = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_pca,y,lr,5), Cross_val(x_spca,y,lr,5), Cross_val(x_bpca,y,lr,5)],axis=1)
result.columns = ["PCA_Train","PCA_Test","sPCA_Train","sPCA_Test","bPCA_Train","bPCA_Test"]
result

,PCA_Train,PCA_Test,sPCA_Train,sPCA_Test,bPCA_Train,bPCA_Test
Average,0.264764,0.254550,0.101188,0.082967,0.264898,0.252538
1th Fold,0.252913,0.312764,0.103265,0.090706,0.301688,0.102650
2th Fold,0.249632,0.303631,0.081073,0.144813,0.262642,0.256598
3th Fold,0.254125,0.296751,0.125240,-0.024663,0.247518,0.323509
4th Fold,0.285728,0.163033,0.095005,0.113321,0.246177,0.329187
5th Fold,0.281421,0.196569,0.101357,0.090657,0.266463,0.250744


In [16]:
sparsity = 50
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
oPCA = OrdinaryEig()
x_pca = pd.DataFrame(np.matmul(X_sparse.values,oPCA.fit_transform(X_sparse.corr()).values))
sEig = SparseEig(r=2,k=10)
x_spca = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
bEig = BinaryEig(r=2,k=[-0.5,0.3])
x_bpca = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_pca,y,lr,5), Cross_val(x_spca,y,lr,5), Cross_val(x_bpca,y,lr,5)],axis=1)
result.columns = ["PCA_Train","PCA_Test","sPCA_Train","sPCA_Test","bPCA_Train","bPCA_Test"]
result

,PCA_Train,PCA_Test,sPCA_Train,sPCA_Test,bPCA_Train,bPCA_Test
Average,0.194163,0.176875,0.047568,0.038148,0.182317,0.167661
1th Fold,0.181358,0.246272,0.048658,0.039477,0.210100,0.040408
2th Fold,0.177915,0.231482,0.040656,0.058803,0.183396,0.169469
3th Fold,0.185609,0.212794,0.048204,0.036262,0.189932,0.143369
4th Fold,0.197237,0.172058,0.046620,0.039862,0.179771,0.184319
5th Fold,0.228696,0.021769,0.053702,0.016337,0.148384,0.300739


In [23]:
sparsity = 80
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
oPCA = OrdinaryEig()
x_pca = pd.DataFrame(np.matmul(X_sparse.values,oPCA.fit_transform(X_sparse.corr()).values))
sEig = SparseEig(r=2,k=10)
x_spca = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
bEig = BinaryEig(r=2,k=[0.3,0.3])
x_bpca = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_pca,y,lr,5), Cross_val(x_spca,y,lr,5), Cross_val(x_bpca,y,lr,5)],axis=1)
result.columns = ["PCA_Train","PCA_Test","sPCA_Train","sPCA_Test","bPCA_Train","bPCA_Test"]
result

,PCA_Train,PCA_Test,sPCA_Train,sPCA_Test,bPCA_Train,bPCA_Test
Average,0.123317,0.111189,0.088924,0.068445,0.071097,0.057961
1th Fold,0.105128,0.205186,0.065563,0.181542,0.062833,0.099126
2th Fold,0.118144,0.123672,0.103783,0.014466,0.063831,0.084207
3th Fold,0.130062,0.083358,0.074401,0.130365,0.089447,-0.026448
4th Fold,0.142701,0.017951,0.108306,-0.020304,0.079469,0.025234
5th Fold,0.120551,0.125776,0.092569,0.036157,0.059908,0.107684


In [24]:
n=500;p=20
X,y = kqc_custom.generate_dependent_sample(n,p,beta_coef=[5 for i in range(8)],epsilon=20,covariance_parameter=5)

In [29]:
Cross_val(pd.DataFrame(X),y,lr,5)
k_schedule=-0.3

In [30]:
sparsity = 20
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
oPCA = OrdinaryEig()
x_pca = pd.DataFrame(np.matmul(X_sparse.values,oPCA.fit_transform(X_sparse.corr()).values))
sEig = SparseEig(r=2,k=10)
x_spca = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bpca = pd.DataFrame(np.matmul(X_sparse.values,bEig_coef))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_pca,y,lr,5), Cross_val(x_spca,y,lr,5), Cross_val(x_bpca,y,lr,5)],axis=1)
result.columns = ["PCA_Train","PCA_Test","sPCA_Train","sPCA_Test","bPCA_Train","bPCA_Test"]
result

,PCA_Train,PCA_Test,sPCA_Train,sPCA_Test,bPCA_Train,bPCA_Test
Average,0.053637,0.047028,0.198092,0.179981,0.510335,0.505776
1th Fold,0.058210,0.021157,0.181582,0.236670,0.505167,0.524683
2th Fold,0.061828,0.014094,0.201923,0.175300,0.544272,0.374379
3th Fold,0.042129,0.097026,0.209185,0.122584,0.523095,0.434863
4th Fold,0.059507,0.020573,0.197855,0.190437,0.499300,0.539283
5th Fold,0.046512,0.082290,0.199915,0.174917,0.479841,0.655672


In [31]:
sparsity = 50
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
oPCA = OrdinaryEig()
x_pca = pd.DataFrame(np.matmul(X_sparse.values,oPCA.fit_transform(X_sparse.corr()).values))
sEig = SparseEig(r=2,k=10)
x_spca = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bpca = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_pca,y,lr,5), Cross_val(x_spca,y,lr,5), Cross_val(x_bpca,y,lr,5)],axis=1)
result.columns = ["PCA_Train","PCA_Test","sPCA_Train","sPCA_Test","bPCA_Train","bPCA_Test"]
result

,PCA_Train,PCA_Test,sPCA_Train,sPCA_Test,bPCA_Train,bPCA_Test
Average,0.029266,0.019142,0.095034,0.076972,0.410775,0.397872
1th Fold,0.031350,0.012629,0.104690,0.034108,0.421734,0.363451
2th Fold,0.031408,0.011160,0.072783,0.154162,0.391820,0.467557
3th Fold,0.034741,-0.013364,0.106618,0.013289,0.419750,0.353500
4th Fold,0.026679,0.024696,0.098981,0.073106,0.402717,0.426262
5th Fold,0.022151,0.060590,0.092097,0.110198,0.417855,0.378589


In [32]:
sparsity = 80
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
oPCA = OrdinaryEig()
x_pca = pd.DataFrame(np.matmul(X_sparse.values,oPCA.fit_transform(X_sparse.corr()).values))
sEig = SparseEig(r=2,k=10)
x_spca = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bpca = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_pca,y,lr,5), Cross_val(x_spca,y,lr,5), Cross_val(x_bpca,y,lr,5)],axis=1)
result.columns = ["PCA_Train","PCA_Test","sPCA_Train","sPCA_Test","bPCA_Train","bPCA_Test"]
result

AttributeError: 'int' object has no attribute 'num_qubits'

In [ ]:
n=500;p=20
X,y = kqc_custom.generate_dependent_sample(n,p,beta_coef=[6,6,5,5,4,4,3,3],epsilon=20,covariance_parameter=5)

In [ ]:
sparsity = 20
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
oPCA = OrdinaryEig()
x_pca = pd.DataFrame(np.matmul(X_sparse.values,oPCA.fit_transform(X_sparse.corr()).values))
sEig = SparseEig(r=2,k=10)
x_spca = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bpca = pd.DataFrame(np.matmul(X_sparse.values,bEig_coef))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_pca,y,lr,5), Cross_val(x_spca,y,lr,5), Cross_val(x_bpca,y,lr,5)],axis=1)
result.columns = ["PCA_Train","PCA_Test","sPCA_Train","sPCA_Test","bPCA_Train","bPCA_Test"]
result

In [ ]:
sparsity = 50
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
oPCA = OrdinaryEig()
x_pca = pd.DataFrame(np.matmul(X_sparse.values,oPCA.fit_transform(X_sparse.corr()).values))
sEig = SparseEig(r=2,k=10)
x_spca = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bpca = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_pca,y,lr,5), Cross_val(x_spca,y,lr,5), Cross_val(x_bpca,y,lr,5)],axis=1)
result.columns = ["PCA_Train","PCA_Test","sPCA_Train","sPCA_Test","bPCA_Train","bPCA_Test"]
result

In [ ]:
sparsity = 80
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
oPCA = OrdinaryEig()
x_pca = pd.DataFrame(np.matmul(X_sparse.values,oPCA.fit_transform(X_sparse.corr()).values))
sEig = SparseEig(r=2,k=10)
x_spca = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bpca = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_pca,y,lr,5), Cross_val(x_spca,y,lr,5), Cross_val(x_bpca,y,lr,5)],axis=1)
result.columns = ["PCA_Train","PCA_Test","sPCA_Train","sPCA_Test","bPCA_Train","bPCA_Test"]
result

In [ ]:
sparsity = 20
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
sirmat = SIRmat(y,X_sparse,4)
oEig = OrdinaryEig()
x_sir = pd.DataFrame(np.matmul(X_sparse.values,oEig.fit_transform(pd.DataFrame(sirmat))))
sEig = SparseEig(r=2,k=10)
x_ssir = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(sirmat))))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bsir = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(sirmat))))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_sir,y,lr,5), Cross_val(x_ssir,y,lr,5), Cross_val(x_bsir,y,lr,5)],axis=1)
result.columns = ["SIR_Train","SIR_Test","sSIR_Train","sSIR_Test","bSIR_Train","bSIR_Test"]
result

In [ ]:
sparsity = 50
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
sirmat = SIRmat(y,X_sparse,4)
oEig = OrdinaryEig()
x_sir = pd.DataFrame(np.matmul(X_sparse.values,oEig.fit_transform(pd.DataFrame(sirmat))))
sEig = SparseEig(r=2,k=10)
x_ssir = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(sirmat))))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bsir = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(sirmat))))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_sir,y,lr,5), Cross_val(x_ssir,y,lr,5), Cross_val(x_bsir,y,lr,5)],axis=1)
result.columns = ["SIR_Train","SIR_Test","sSIR_Train","sSIR_Test","bSIR_Train","bSIR_Test"]
result

In [ ]:
sparsity = 80
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
sirmat = SIRmat(y,X_sparse,4)
oEig = OrdinaryEig()
x_sir = pd.DataFrame(np.matmul(X_sparse.values,oEig.fit_transform(pd.DataFrame(sirmat))))
sEig = SparseEig(r=2,k=10)
x_ssir = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(sirmat))))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bsir = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(sirmat))))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_sir,y,lr,5), Cross_val(x_ssir,y,lr,5), Cross_val(x_bsir,y,lr,5)],axis=1)
result.columns = ["SIR_Train","SIR_Test","sSIR_Train","sSIR_Test","bSIR_Train","bSIR_Test"]
result

In [ ]:
n=500;p=20
X,y = kqc_custom.generate_dependent_sample(n,p,beta_coef=[6,6,5,5,4,4,3,3],epsilon=30,covariance_parameter=5)

In [ ]:
sparsity = 20
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
sirmat = SIRmat(y,X_sparse,4)
oEig = OrdinaryEig()
x_sir = pd.DataFrame(np.matmul(X_sparse.values,oEig.fit_transform(pd.DataFrame(sirmat))))
sEig = SparseEig(r=2,k=10)
x_ssir = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(sirmat))))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bsir = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(sirmat))))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_sir,y,lr,5), Cross_val(x_ssir,y,lr,5), Cross_val(x_bsir,y,lr,5)],axis=1)
result.columns = ["SIR_Train","SIR_Test","sSIR_Train","sSIR_Test","bSIR_Train","bSIR_Test"]
result

In [ ]:
sparsity = 50
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
sirmat = SIRmat(y,X_sparse,4)
oEig = OrdinaryEig()
x_sir = pd.DataFrame(np.matmul(X_sparse.values,oEig.fit_transform(pd.DataFrame(sirmat))))
sEig = SparseEig(r=2,k=10)
x_ssir = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(sirmat))))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bsir = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(sirmat))))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_sir,y,lr,5), Cross_val(x_ssir,y,lr,5), Cross_val(x_bsir,y,lr,5)],axis=1)
result.columns = ["SIR_Train","SIR_Test","sSIR_Train","sSIR_Test","bSIR_Train","bSIR_Test"]
result

In [ ]:
sparsity = 80
X_sparse = pd.DataFrame(X*(np.random.randint(0,100,n*p) > sparsity).reshape((n,p)))
sirmat = SIRmat(y,X_sparse,4)
oEig = OrdinaryEig()
x_sir = pd.DataFrame(np.matmul(X_sparse.values,oEig.fit_transform(pd.DataFrame(sirmat))))
sEig = SparseEig(r=2,k=10)
x_ssir = pd.DataFrame(np.matmul(X_sparse.values,sEig.fit_transform(pd.DataFrame(sirmat))))
bEig = BinaryEig(r=2,k=k_schedule)
bEig_coef = bEig.fit_transform(pd.DataFrame(X_sparse).corr()).values
x_bsir = pd.DataFrame(np.matmul(X_sparse.values,bEig.fit_transform(pd.DataFrame(sirmat))))
lr = LinearRegression()
k=5
result = pd.concat([Cross_val(x_sir,y,lr,5), Cross_val(x_ssir,y,lr,5), Cross_val(x_bsir,y,lr,5)],axis=1)
result.columns = ["SIR_Train","SIR_Test","sSIR_Train","sSIR_Test","bSIR_Train","bSIR_Test"]
result